In [1]:
import config
import data
import os

from transformers import TFAutoModel
import tensorflow as tf

from keras.models import Model
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import (
    Concatenate,
    Conv1D,
    Dense,
    Dropout,
    # Embedding,
    Flatten,
    Input,
    MaxPooling1D,
)

import json
import numpy as np
import pprint


from training_functions import (
    # calculate_estimated_year_tensor,
    custom_loss,
    # custom_objects_dict,
    custom_metric,
    evaluate_model,
    prediction,
    save_accuracy_by_interval_and_gender,
    save_hist_confusion_matrix,
    save_training_history,
)

In [2]:
def mm_cnn(
    num_date_classes:int ,
    model_id: str=None,
    max_length: int=514,
    dense_units: int=16,
    conv_filters: int=32, 
    conv_kernel_size: int=3, 
):
    """
    Crée un modèle Keras avec un modèle BERT pré-entraîné pour une tâche multitâche de classification.

    Parameters:
    - model_id: str, identifiant du modèle pré-entraîné à utiliser (par exemple, 'bert-base-uncased')
    - max_length: int, la longueur maxiHomme des séquences d'entrée
    - dense_units: int, nombre d'unités pour les couches denses individuelles
    - concat_dense_units: int, nombre d'unités pour la couche dense après concaténation

    Returns:
    - model: Keras Model, le modèle compilé
    """
    # Charger le modèle BERT pré-entraîné
    bert_model = TFAutoModel.from_pretrained(model_id)

    # Définir les entrées du modèle
    input_ids = Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask")

    # Passer les entrées dans le modèle BERT
    bert_output = bert_model(input_ids, attention_mask=attention_mask) 
    sequence_output = bert_output.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)  Utiliser ensuite avec un CNN
    # pooled_output = bert_output.pooler_output  # Shape: (batch_size, hidden_size) # Utiliser le pooled_output pour la classification

    # CNN
    conv_layer = Conv1D(filters=conv_filters, kernel_size=conv_kernel_size, activation="relu", name="Conv1D")(sequence_output)
    pooling_layer = MaxPooling1D(pool_size=2, name="MaxPooling1D")(conv_layer)
    flatten_layer = Flatten(name="Flatten")(pooling_layer)
    dropout_layer = Dropout(0.3, name="Dropout")(flatten_layer) # Ajout des couches supplémentaires pour les tâches spécifiques

    # Dense layers for individual tasks
    dense_layer = Dense(units=dense_units, activation="relu", name="Dense")(dropout_layer)

    # Output layers for individual tasks
    sexe_output = Dense(1, activation="sigmoid", name="Sexe_output")(dense_layer)
    date_output = Dense(num_date_classes, activation="softmax", name="Date_output")(dense_layer)

    # Créer le modèle
    model = Model(inputs=[input_ids, attention_mask], outputs=[sexe_output, date_output])

    return model

In [3]:
# 1 : Charger les données d'entraînements et de validation
inputs_and_labels = data.main()

train_inputs = inputs_and_labels["train_inputs"]
train_sexe_labels = inputs_and_labels["train_sexe_labels"]
train_date_labels = inputs_and_labels["train_date_labels"]

val_inputs = inputs_and_labels["val_inputs"]
val_sexe_labels = inputs_and_labels["val_sexe_labels"]
val_date_labels = inputs_and_labels["val_date_labels"]

# Libérer la mémoire occupée par inputs_and_labels
del inputs_and_labels

# Créer le répertoire s'il n'existe pas
os.makedirs(config.MM_CNN_RESULT_PATH, exist_ok=True)

# 1. Intialiser le modèle
model = mm_cnn(num_date_classes=len(config.DATE_MAP), model_id=config.MODEL_ID)

# 2. Compiler le modèle avec les fonctions de perte appropriées pour chaque sortie
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss={
        "Sexe_output": "binary_crossentropy", 
        "Date_output": custom_loss,
    },
    loss_weights={
        "Sexe_output": config.SEXE_LOSS_WEIGHT,
        "Date_output": config.DATE_LOSS_WEIGHT
    },
    metrics={
        "Sexe_output": "accuracy", 
        "Date_output": custom_metric,
    }
)

# Sauvegarder l'architecture du modèle en .png
plot_model(model=model, show_shapes=True, to_file=config.MM_CNN_RESULT_PATH + f"{config.MM_CNN}_model_arch.png")

# Callbacks
checkpoint_callback = ModelCheckpoint(
    filepath=config.MM_CNN_RESULT_PATH + f"{config.MM_CNN}_best_model.h5",
    monitor="val_loss",
    save_best_only=True,
    mode="min", # Sauvegarder le modèle avec la perte minimale
    verbose=1
)

# Early stopping
early_stopping_callback = EarlyStopping(
    monitor="val_loss",
    patience=config.PATIENCE,
    verbose=1, # Affichage d'un message
    restore_best_weights=True # Restaurer les poids du meilleur modèle après l'arrêt
)

# 3. Entraîner le modèle
history = model.fit(
    x=train_inputs,
    y={
        "Sexe_output": train_sexe_labels, 
        "Date_output": train_date_labels,
        },
    epochs=config.EPOCHS,
    batch_size=config.BATCH_SIZE,
    callbacks=[
        checkpoint_callback, 
        early_stopping_callback,
    ],
    validation_data=(
        val_inputs, 
        {
            "Sexe_output": val_sexe_labels, 
            "Date_output": val_date_labels,
        }),
)
    
# Sauvegarder l'historique de l'entraînement en .json
with open(config.MM_CNN_RESULT_PATH + f"{config.MM_CNN}_history.json", "w") as json_file:
    json.dump(history.history, json_file, indent=4)
# Sauvegarder l'historique de l'entraînement en .png
save_training_history(history_data=history.history, save_dir=config.MM_CNN_RESULT_PATH, base_filename=config.MM_CNN, single_figure=True)
save_training_history(history_data=history.history, save_dir=config.MM_CNN_RESULT_PATH, base_filename=config.MM_CNN, single_figure=False)


# Evaluation --------------------------------


# 1. Charger les données de tests
inputs_and_labels = data.main()

test_inputs = inputs_and_labels["test_inputs"]
test_sexe_labels = inputs_and_labels["test_sexe_labels"]
test_date_labels = inputs_and_labels["test_date_labels"]

# Libérer la mémoire occupée par inputs_and_labels
del inputs_and_labels

# Évaluer le modèle sur les données de test avec le GPU
evaluation_results = model.evaluate(
    x = test_inputs,
    y = {
        "Sexe_output": np.array(test_sexe_labels),
        "Date_output": np.array(test_date_labels),
    },
    return_dict=True
)
    
print("\nEvaluation...\n")
pprint.pp(evaluation_results)

# Sauvegarder les résultats de l'évaluation
with open(config.MM_CNN_RESULT_PATH + f"{config.MM_CNN}_evaluation_results.json", "w") as json_file:
    json.dump(evaluation_results, json_file, indent=4)
    
# Charger le modèle pré entraîné
# model = load_model(
#     filepath=config.MM_CNN_RESULT_PATH + f"{config.MM_CNN}_best_model.h5", 
#     custom_objects=custom_objects_dict()
# )

print("\nPrédictions...\n")
prediction_df = prediction(model=model, input_data=test_inputs, sexe_label=test_sexe_labels, date_label=test_date_labels)

evaluate_model(
    df=prediction_df, 
    confusion_matrix_output=config.MM_CNN_RESULT_PATH + f"{config.MM_CNN}_confusion_matrix.png", 
    roc_curve_output=config.MM_CNN_RESULT_PATH + f"{config.MM_CNN}_roc_curve.png"
)

prediction_df["interval"] = prediction_df["true date"].map(config.DATE_MAP)

save_hist_confusion_matrix(df=prediction_df, output_file=config.MM_CNN_RESULT_PATH + f"{config.MM_CNN}_hist_confusion_matrix.png")

save_accuracy_by_interval_and_gender(df=prediction_df, output_file=config.MM_CNN_RESULT_PATH + f"{config.MM_CNN}_accuracy_by_interval_sexes.png")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFCamembertModel: ['lm_head.layer_norm.weight', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing TFCamembertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFCamembertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFCamembertModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions 

Epoch 1/3
10/10 [==============================] - ETA: 0s - loss: 1.6129 - Sexe_output_loss: 0.9592 - Date_output_loss: 6537.6025 - Sexe_output_accuracy: 0.4901 - Date_output_custom_metric: 0.2188
Epoch 1: val_loss improved from inf to 1.39075, saving model to ../results/mm_cnn\mm_cnn_best_model.h5
10/10 [==============================] - 21s 1s/step - loss: 1.6129 - Sexe_output_loss: 0.9592 - Date_output_loss: 6537.6025 - Sexe_output_accuracy: 0.4901 - Date_output_custom_metric: 0.2188 - val_loss: 1.3908 - val_Sexe_output_loss: 0.7452 - val_Date_output_loss: 6455.5781 - val_Sexe_output_accuracy: 0.4509 - val_Date_output_custom_metric: 0.2014
Epoch 2/3
10/10 [==============================] - ETA: 0s - loss: 1.3731 - Sexe_output_loss: 0.7170 - Date_output_loss: 6561.0991 - Sexe_output_accuracy: 0.5695 - Date_output_custom_metric: 0.2348
Epoch 2: val_loss did not improve from 1.39075
Restoring model weights from the end of the best epoch: 1.
10/10 [==============================] - 9s 

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
c:\Users\Vincent\Documents\Cours\M2\Modélisation de système de vision\Projet\projet-MSV\scripts\training_functions.py:193: RuntimeWarning: invalid value encountered in scalar divide
  accuracy = correct_predictions / total
c:\Users\Vincent\Documents\Cours\M2\Modélisation de système de vision\Projet\projet-MSV\scripts\training_functions.py:193: RuntimeWarning: invalid value encountered in scalar divide
  accuracy = correct_predictions / total


In [4]:
import config
import data
import os

from transformers import TFAutoModel
import tensorflow as tf

from keras.models import Model
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import (
    Concatenate,
    Conv1D,
    Dense,
    Dropout,
    # Embedding,
    Flatten,
    Input,
    MaxPooling1D,
)

import json
import numpy as np
import pprint


from training_functions import (
    # calculate_estimated_year_tensor,
    custom_loss,
    # custom_objects_dict,
    custom_metric,
    evaluate_model,
    prediction,
    save_accuracy_by_interval_and_gender,
    save_hist_confusion_matrix,
    save_training_history,
)



def ss_cnn(
    model_id: str=None,
    max_length: int=514,
    dense_units: int=16,
    conv_filters: int=32, 
    conv_kernel_size: int=3, 
):
    """
    Crée un modèle Keras avec un modèle BERT pré-entraîné pour une tâche de classification du sexe.

    Parameters:
    - model_id: str, identifiant du modèle pré-entraîné à utiliser (par exemple, 'bert-base-uncased')
    - max_length: int, la longueur maxiHomme des séquences d'entrée
    - dense_units: int, nombre d'unités pour les couches denses
    - conv_filters: int, nombre de filtres pour la couche Conv1D
    - conv_kernel_size: int, taille du noyau pour la couche Conv1D

    Returns:
    - model: Keras Model, le modèle compilé
    """
    # Charger le modèle BERT pré-entraîné
    bert_model = TFAutoModel.from_pretrained(model_id)

    # Définir les entrées du modèle
    input_ids = Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask")

    # Passer les entrées dans le modèle BERT
    bert_output = bert_model(input_ids, attention_mask=attention_mask)
    sequence_output = bert_output.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)

    # CNN
    conv_layer = Conv1D(filters=conv_filters, kernel_size=conv_kernel_size, activation="relu", name="Conv1D")(sequence_output)
    pooling_layer = MaxPooling1D(pool_size=2, name="MaxPooling1D")(conv_layer)
    flatten_layer = Flatten(name="Flatten")(pooling_layer)
    dropout_layer = Dropout(0.3, name="Dropout")(flatten_layer)

    # Dense layer for sex classification
    dense_layer_sexe = Dense(units=dense_units, activation="relu", name="Dense_sexe")(dropout_layer)

    # Output layer for sex classification
    sexe_output = Dense(1, activation="sigmoid", name="Sexe_output")(dense_layer_sexe)

    # Créer le modèle
    model = Model(inputs=[input_ids, attention_mask], outputs=sexe_output)

    return model


def main():
    # 1 : Charger les données d'entraînements et de validation
    inputs_and_labels = data.main()

    train_inputs = inputs_and_labels["train_inputs"]
    train_sexe_labels = inputs_and_labels["train_sexe_labels"]
    # train_date_labels = inputs_and_labels["train_date_labels"]

    val_inputs = inputs_and_labels["val_inputs"]
    val_sexe_labels = inputs_and_labels["val_sexe_labels"]
    # val_date_labels = inputs_and_labels["val_date_labels"]

    # Libérer la mémoire occupée par inputs_and_labels
    del inputs_and_labels

    # Créer le répertoire s'il n'existe pas
    os.makedirs(config.SS_CNN_RESULT_PATH, exist_ok=True)

    # 1. Intialiser le modèle
    model = ss_cnn(model_id=config.MODEL_ID)

    # 2. Compiler le modèle avec les fonctions de perte appropriées pour chaque sortie
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
        loss={
            "Sexe_output": "binary_crossentropy", 
            # "Date_output": custom_loss,
        },
        loss_weights={
            "Sexe_output": config.SEXE_LOSS_WEIGHT,
            # "Date_output": config.DATE_LOSS_WEIGHT
        },
        metrics={
            "Sexe_output": "accuracy", 
            # "Date_output": custom_metric,
        }
    )

    # Sauvegarder l'architecture du modèle en .png
    plot_model(model=model, show_shapes=True, to_file=config.SS_CNN_RESULT_PATH + f"{config.SS_CNN}_model_arch.png")

    # Callbacks
    checkpoint_callback = ModelCheckpoint(
        filepath=config.SS_CNN_RESULT_PATH + f"{config.SS_CNN}_best_model.h5",
        monitor="val_loss",
        save_best_only=True,
        mode="min", # Sauvegarder le modèle avec la perte minimale
        verbose=1
    )

    # Early stopping
    early_stopping_callback = EarlyStopping(
        monitor="val_loss",
        patience=config.PATIENCE,
        verbose=1, # Affichage d'un message
        restore_best_weights=True # Restaurer les poids du meilleur modèle après l'arrêt
    )

    # 3. Entraîner le modèle
    print("\nDébut entraînement ss_cnn\n")
    history = model.fit(
        x=train_inputs,
        y={
            "Sexe_output": train_sexe_labels, 
            # "Date_output": train_date_labels,
            },
        epochs=config.EPOCHS,
        batch_size=config.BATCH_SIZE,
        callbacks=[
            checkpoint_callback, 
            early_stopping_callback,
        ],
        validation_data=(
            val_inputs, 
            {
                "Sexe_output": val_sexe_labels, 
                # "Date_output": val_date_labels,
            }),
    )
    print("\Fin entraînement ss_cnn\n")
        
    # Sauvegarder l'historique de l'entraînement en .json
    with open(config.SS_CNN_RESULT_PATH + f"{config.SS_CNN}_history.json", "w") as json_file:
        json.dump(history.history, json_file, indent=4)
    # Sauvegarder l'historique de l'entraînement en .png
    save_training_history(history_data=history.history, save_dir=config.SS_CNN_RESULT_PATH, base_filename=config.SS_CNN, single_figure=True)
    save_training_history(history_data=history.history, save_dir=config.SS_CNN_RESULT_PATH, base_filename=config.SS_CNN, single_figure=False)


    # Evaluation --------------------------------


    # 1. Charger les données de tests
    inputs_and_labels = data.main()

    test_inputs = inputs_and_labels["test_inputs"]
    test_sexe_labels = inputs_and_labels["test_sexe_labels"]
    test_date_labels = inputs_and_labels["test_date_labels"]

    # Libérer la mémoire occupée par inputs_and_labels
    del inputs_and_labels

    # Évaluer le modèle sur les données de test avec le GPU
    evaluation_results = model.evaluate(
        x = test_inputs,
        y = {
            "Sexe_output": np.array(test_sexe_labels),
            # "Date_output": np.array(test_date_labels),
        },
        return_dict=True
    )
        
    print("\nEvaluation...\n")
    pprint.pp(evaluation_results)

    # Sauvegarder les résultats de l'évaluation
    with open(config.SS_CNN_RESULT_PATH + f"{config.SS_CNN}_evaluation_results.json", "w") as json_file:
        json.dump(evaluation_results, json_file, indent=4)
        
    # Charger le modèle pré entraîné
    # model = load_model(
    #     filepath=config.SS_CNN_RESULT_PATH + f"{config.SS_CNN}_best_model.h5", 
    #     custom_objects=custom_objects_dict()
    # )

    print("\nPrédictions...\n")
    prediction_df = prediction(model=model, input_data=test_inputs, sexe_label=test_sexe_labels, date_label=None)

    evaluate_model(
        df=prediction_df, 
        confusion_matrix_output=config.SS_CNN_RESULT_PATH + f"{config.SS_CNN}_confusion_matrix.png", 
        roc_curve_output=config.SS_CNN_RESULT_PATH + f"{config.SS_CNN}_roc_curve.png"
    )

    prediction_df["interval"] = prediction_df["true date"].map(config.DATE_MAP)

    save_hist_confusion_matrix(df=prediction_df, output_file=config.SS_CNN_RESULT_PATH + f"{config.SS_CNN}_hist_confusion_matrix.png")

    save_accuracy_by_interval_and_gender(df=prediction_df, output_file=config.SS_CNN_RESULT_PATH + f"{config.SS_CNN}_accuracy_by_interval_sexes.png")

In [5]:
main()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFCamembertModel: ['lm_head.layer_norm.weight', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing TFCamembertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFCamembertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFCamembertModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions 


Début entraînement ss_cnn

Epoch 1/3
10/10 [==============================] - ETA: 0s - loss: 0.8506 - accuracy: 0.4901
Epoch 1: val_loss improved from inf to 0.77274, saving model to ../results/ss_cnn\ss_cnn_best_model.h5
10/10 [==============================] - 24s 2s/step - loss: 0.8506 - accuracy: 0.4901 - val_loss: 0.7727 - val_accuracy: 0.4909
Epoch 2/3
10/10 [==============================] - ETA: 0s - loss: 0.6789 - accuracy: 0.5430
Epoch 2: val_loss improved from 0.77274 to 0.70497, saving model to ../results/ss_cnn\ss_cnn_best_model.h5
10/10 [==============================] - 11s 1s/step - loss: 0.6789 - accuracy: 0.5430 - val_loss: 0.7050 - val_accuracy: 0.5200
Epoch 3/3
 2/10 [=====>........................] - ETA: 4s - loss: 0.6368 - accuracy: 0.5938

KeyboardInterrupt: 